<a href="https://colab.research.google.com/github/KirillKonovalov/fun/blob/master/Sn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
!pip install dgl

     |████████████████████████████████| 3.6MB 5.8MB/s 


In [ ]:
import numpy as np

In [ ]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
labeled_edges = np.loadtxt('/gdrive/My Drive/social_network/labeled_edges.txt', dtype=int)
labeled_edges

array([[2495,  598,    0],
       [1473, 1570,    1],
       [1000,  748,    1],
       ...,
       [1816, 2097,    0],
       [ 218,  876,    0],
       [1985, 1657,    0]])

In [ ]:
unlabeled_edges = np.loadtxt('/gdrive/My Drive/social_network/unlabeled_edges.txt', dtype=int)
unlabeled_edges

array([[1429, 2808],
       [1125, 1250],
       [2277, 2810],
       ...,
       [ 202,  387],
       [1284, 1017],
       [1958, 2407]])

In [ ]:
features = np.loadtxt('/gdrive/My Drive/social_network/features.txt', dtype=int)
features

array([[   0,    0,    0, ...,    0,    0,    0],
       [   1,    0,    0, ...,    0,    1,    0],
       [   2,    0,    1, ...,    0,    0,    0],
       ...,
       [3309,    0,    0, ...,    0,    1,    0],
       [3310,    0,    0, ...,    0,    0,    0],
       [3311,    0,    0, ...,    0,    0,    1]])

In [ ]:
features.shape

(3312, 3710)

In [ ]:
import dgl

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [ ]:
features.shape[1]

3710

In [ ]:
features.shape[0]

3312

In [ ]:
graph = dgl.DGLGraph()
graph.add_nodes(3312)
graph.add_edges(labeled_edges[:, 0], labeled_edges[:, 1])
graph.add_edges(unlabeled_edges[:, 0], unlabeled_edges[:, 1])

/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [ ]:
import torch
import torch.nn as nn

In [ ]:
feature_tensor = torch.Tensor(features[:, 1:])
feature_tensor.shape


torch.Size([3312, 3709])

In [ ]:
graph.edges()[0].shape

torch.Size([9072])

In [ ]:
test_mask = np.random.choice(np.arange(6048), size=300, replace=False)

In [ ]:
train_mask = np.array(list(set(np.arange(6048)).difference(test_mask)))

In [ ]:
graph.edges()[0][train_mask], graph.edges()[1][train_mask]

(tensor([2495, 1473, 1000,  ..., 1816,  218, 1985]),
 tensor([ 598, 1570,  748,  ..., 2097,  876, 1657]))

In [ ]:
labels = torch.as_tensor(labeled_edges[:, 2])
labels

tensor([0, 1, 1,  ..., 0, 0, 0])

In [ ]:
g, feature_tensor, labels, train_mask, test_mask = graph, feature_tensor, labeled_edges[:, 2], train_mask, test_mask

In [ ]:
g = dgl.add_self_loop(g)

In [ ]:
from dgl.nn.pytorch import GraphConv
import torch.nn.functional as F

In [ ]:
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.layer1 = GraphConv(3709, 128)
        self.layer2 = GraphConv(128, 64)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x

In [ ]:
net = GCN()
net.forward(g, feature_tensor)

tensor([[ 0.0127,  0.0145, -0.0189,  ...,  0.0010, -0.0095,  0.0024],
        [ 0.0145, -0.0139, -0.0251,  ..., -0.0624, -0.0395, -0.0488],
        [ 0.0081,  0.0162, -0.0205,  ..., -0.0128,  0.0114, -0.0126],
        ...,
        [ 0.0484, -0.0032, -0.0227,  ..., -0.0668,  0.0227,  0.0509],
        [ 0.0061,  0.0500, -0.0264,  ..., -0.0677, -0.0119, -0.0767],
        [ 0.0478,  0.0575, -0.0389,  ...,  0.0085, -0.0093, -0.0302]],
       grad_fn=<AddBackward0>)

In [ ]:
import dgl.function as fn

In [ ]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.gcn = GCN()
        self.output = nn.Linear(64, 2)
    
    def forward(self, g, features):
        h = self.gcn.forward(g, features)
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(fn.u_mul_v(lhs_field='h', rhs_field='h', out='score'))
            score = g.edata['score']
            out = self.output(score)
        return out

In [ ]:
net = model()

In [ ]:
predictions = net.forward(g, feature_tensor)
predictions

tensor([[ 0.0354, -0.1090],
        [ 0.0350, -0.1194],
        [ 0.0337, -0.1222],
        ...,
        [ 0.0323, -0.1216],
        [ 0.0334, -0.1200],
        [ 0.0363, -0.1241]], grad_fn=<AddmmBackward>)

In [ ]:
predictions.shape

torch.Size([12384, 2])

In [ ]:
predictions[train_mask]

tensor([[ 0.0354, -0.1090],
        [ 0.0350, -0.1194],
        [ 0.0337, -0.1222],
        ...,
        [ 0.0365, -0.1214],
        [ 0.0360, -0.1178],
        [ 0.0376, -0.1205]], grad_fn=<IndexBackward>)

In [ ]:
predictions[train_mask].shape

torch.Size([5748, 2])

In [ ]:
answer_mask = np.arange(6048, 9072)

In [ ]:
predictions[answer_mask].argmax(dim=1)

tensor([0, 0, 0,  ..., 0, 0, 0])

In [ ]:
labels[train_mask]

array([0, 1, 1, ..., 0, 0, 0])

In [ ]:
labels[train_mask].shape

(5748,)

In [ ]:
loss = F.cross_entropy(predictions[train_mask], labels[train_mask])
#loss.backward()
loss

TypeError: ignored

In [ ]:
with torch.no_grad():
    loss = F.cross_entropy(predictions[test_mask], labels[test_mask])
loss

tensor(0.6956)

In [ ]:
loss

tensor(0.6956)